### Utilities to Upload a CSV Dataset to SQL

To use the Python function we provided here, you need to have
- a csv file with the data you want to upload. Here we show you with an exampe, `./covid-19/WHO-COVID-19-global-data.csv`.
- Decide the database you want to upload the data to, and the name of the table you want to host them. You don't need to worry if the database and table exist, the function we provide here we take care of no matter you have them or not.

When you use this tool for your own data, change the cell below. A good way to organize is to save the data for different interviews in different databases to prevent mixing data.

In [1]:
csv_file = "./covid-19/WHO-COVID-19-global-data.csv"
table = "who_data"
database = "covid"

**Below we first load the csv file to a pandas data frame.** Notice that SQL doesn't allow spaces in column names so the `clear_col` function replaces spaces with underscores. If you find any other problem in your column names, modify this function clean them.

In [2]:
import numpy as np
import pandas as pd

def clear_cols(s):
    return s.strip().replace(" ", "_")

Below we load the data and make quickly check if the column names are legal.

In [5]:
df = pd.read_csv(csv_file).rename(columns=clear_cols)
df.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-02-24,AF,Afghanistan,EMRO,5,5,0,0
1,2020-02-25,AF,Afghanistan,EMRO,0,5,0,0
2,2020-02-26,AF,Afghanistan,EMRO,0,5,0,0
3,2020-02-27,AF,Afghanistan,EMRO,0,5,0,0
4,2020-02-28,AF,Afghanistan,EMRO,0,5,0,0


Below we generate the command that checks if the database you are to use exists. If not, it will create one. Check the SQL command it outputs below.

In [6]:
def go_db(db):
    cmd =  "\n".join([
        "CREATE DATABASE IF NOT EXISTS {db};", 
        "USE {db};"])
    cmd = cmd.format(db=db)
    
    return cmd 

print(go_db(database))

CREATE DATABASE IF NOT EXISTS covid;
USE covid;


Below we generate the command that checks if the table you are to use exists. If not, it will create one. We create a very simple, possibly oversimplified schema. Keep in mind that we build the table for practicing only. Check the SQL command it outputs below.


In [7]:
def create_schema(df):
    schema = str(df.dtypes).replace(
        "dtype: object", ""
    ).replace(
        "object", "TEXT"
    ).replace(
        "int64", "bigint(20) DEFAULT NULL"
    ).replace(
        "float64", "double DEFAULT NULL"
    )
    
    schema = ",\n".join([
        s.strip() for s in schema.split("\n") if s.strip()])
    
    return schema

def create_table(tname, df):
    schema = create_schema(df)
    
    cmd = """
    CREATE TABLE IF NOT EXISTS {tname} (\n{schema}\n);
    """.format(tname=tname, schema=schema).strip()
    
    return cmd

print(create_table(table, df))

CREATE TABLE IF NOT EXISTS who_data (
Date_reported        TEXT,
Country_code         TEXT,
Country              TEXT,
WHO_region           TEXT,
New_cases             bigint(20) DEFAULT NULL,
Cumulative_cases      bigint(20) DEFAULT NULL,
New_deaths            bigint(20) DEFAULT NULL,
Cumulative_deaths     bigint(20) DEFAULT NULL
);


Below we generate the command that uploads data. People probably remember `LOAD DATA LOCAL INFIL` command from our SQL class. But that commands requires a few setting. Inserting line by line turns out to be an easier way to load data. 

When you use the code below you might find uploading quite easy but it's actually not. We need to generate as a very long sequence of code and take care of many details of it. Check out the code below and the first 300 characters of the generated command below:

In [8]:
def clean_vals(x):
    if isinstance(x, str):
        s = x.replace("\"", "\\\"").replace("\'", "\\\'")
        s = '"' + s + '"'
    elif np.isnan(x):
            s = "NULL"
    else:
        s = str(x)
        
    return s

def gen_values(df):
    values = []
    for row in df.values:
        values.append(
            "(" + ",".join([clean_vals(x) for x in row]) + ")")
        
    return  ",\n".join(values)

def gen_insert(table, df):
    cmd = "INSERT INTO {tname}\nVALUES\n{vals}".format(
        tname=table, vals = gen_values(df))
    
    return cmd

print(gen_insert(table, df)[:300])

INSERT INTO who_data
VALUES
("2020-02-24","AF","Afghanistan","EMRO",5,5,0,0),
("2020-02-25","AF","Afghanistan","EMRO",0,5,0,0),
("2020-02-26","AF","Afghanistan","EMRO",0,5,0,0),
("2020-02-27","AF","Afghanistan","EMRO",0,5,0,0),
("2020-02-28","AF","Afghanistan","EMRO",0,5,0,0),
("2020-02-29","AF","Af


We then save the three steps into a .sql script.

In [9]:
# path = "./myscript.sql"
# script = "\n".join([go_db(database), create_table(table, df), gen_insert(table, df)])
# with open(path, "w") as f:
#     f.write(script)